<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [334]:
import pandas as pd
import psycopg2
import plotly.express as px

# !Данная библиотека используется только для того, чтобы не выводить разные предупреждения и не используется для расчетов!
import warnings
warnings.filterwarnings('ignore')

In [336]:
# Параметры подключения к базе данных
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [337]:
# Запрос считает количество вакансий, используя функцию count()
query_3_1 = f'''
    select 
        count(*) vacancies_count
    from 
        vacancies
'''

In [338]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
display(df)

,vacancies_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [339]:
# Запрос считает количество работодателей, используя функцию count()
query_3_2 = f'''
    select 
        count(*) employments_count
    from 
        employers
'''

In [340]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
display(df)

,employments_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [341]:
# Запрос считает количество регионов в которых размещены вакансии, используя функцию count()
query_3_3 = f'''
    select 
        count(*) regions_count
    from 
        areas
'''

In [342]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
display(df)

,regions_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [343]:
# Запрос считает количество сфер деятельности(спектр услуг в которых работает компания), используя функцию count()
query_3_4 = f'''
    select 
        count(*) industries_count
    from 
        industries
'''

In [344]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
display(df)

,industries_count
0,294


***

# Выводы:

&ensp; Большое количество вакансий может свидетельствовать о высоком спросе на рабочие места, что в свою очередь указывает на экономический рост. 

&ensp; Большое количество работодателей на рынке труда является для соискателей большим плюсом в возможности выбора работы. Это повышает конкуренцию на рынке труда за квалифицированных сотрудников, что может улучшить условия труда и зарплат.

&ensp; Географическое разнообразие вакансий свидетельствует о потребностях в сотрудниках не только в крупных городах, но и в менее населенных регионах, это также является плюсом для соискателей живущих в разных регионах без необходимости поиска работы в крупных городах.

&ensp; Количество сфер деятельности может указывать на многообразие отраслей, в которых компании ищут сотрудников. Это предоставляет возможности для специалистов с различными навыками и квалификациями, а также показывает подвижность рынка труда.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (areas).
Отсортируйте по количеству вакансий в порядке убывания.

In [345]:
# Запрос считает количество вакансий в зависимости от региона, используя функцию count() и group by(). 
# Сортировка по убыванию с помощью order by() desc
query_4_1 = f'''
    select
        a.name areas,
        count(v.id) cnt
    from
        vacancies v
        join areas a on v.area_id = a.id
    group by a.name
    order by cnt desc
'''

In [346]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
display(df)

,areas,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [347]:
# Запрос считает количество вакансий c заполненными полями (минимум одно) с заплатой с помощью count() и фильтров is not null
query_4_2 = f'''
    select
        count(*) filled_salary_count
    from
        vacancies v
    where v.salary_from is not null 
    or v.salary_to is not null
'''

In [348]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
display(df)

,filled_salary_count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [349]:
# Запрос округляет среднюю верхнуюю и нижнюю вилку заработной платы с помощью round(), avg()
query_4_3 = f'''
    select
        round(avg(v.salary_to)) upper_boarder,
        round(avg(v.salary_from)) lower_boarder
    from
        vacancies v
'''

In [350]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
display(df)

,upper_boarder,lower_boarder
0,110537.0,71065.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [351]:
# Запрос выводит строку - тип рабочего графика и типа трудоустройства (через дефиз) с помощью функции format().
# С помощью count() выводится количество вакансий по этим типам используя группировку group by(), далее сортировка по убыванию.
query_4_4 = f'''
    select
        format('%s - %s', schedule, employment) employment_type,
        count(*) vacancies_count
    from
        vacancies v
    group by schedule, employment
    order by vacancies_count desc
'''

In [352]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
display(df)

,employment_type,vacancies_count
0,Полный день - Полная занятость,35367
1,Удаленная работа - Полная занятость,7802
2,Гибкий график - Полная занятость,1593
3,Удаленная работа - Частичная занятость,1312
4,Сменный график - Полная занятость,940
5,Полный день - Стажировка,569
6,Вахтовый метод - Полная занятость,367
7,Полный день - Частичная занятость,347
8,Гибкий график - Частичная занятость,312
9,Полный день - Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [353]:
# Запрос выводит требуемый опыт работы и количество вакансий по нему с помощью count(), группировки group by()
query_4_5 = f'''
    select
        v.experience,
        count(*) vacancies_count
    from
        vacancies v
    group by v.experience
    order by vacancies_count 
'''

In [354]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
display(df)

,experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

# Выводы:

&ensp; Судя по полученным данным детального анализа вакансий, можно сделать вывод, что наибольшее количество вакансий предлагается в мегаполисах России и СНГ, где Москва занимается первое место, что ожидаемо учитывая ее статус крупнейшего экономического центра России и СНГ, второе место Санкт-Петербург, подтверждая свою роль как второго по значимости города в России и закрывает тройку Минск (Белоруссия) - 2,112, что указывает на активный рынок в Белоруссии.

&ensp; Широкая вилка зарплат может свидетельствовать о довольно большом диапазоне зарплат в зависимости от квалификации соискателя. В основном компаниям требуются сотрудники на полный рабочий день (полная занятость), что указывает на предпочтение традиционных графиков работы. Удаленная работа с полной занятостью становится популярнее, однако все еще уступает по количеству вакансий в офисе на полный день. В связи с тем, что работадателям требуются сотрудники имеющие опыт работы от 1 года до 3 лет и от 3 лет до 6 свидетельствует о высоком спросе на специалистов среднего уровня. Однако достаточно высокий спрос на сотрудников без опыта работы свидетельствует об открытости рынка труда для новых специалистов и студентов.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [355]:
# Запрос который определяет первое место по количеству вакансий,
# второй запрос определяет пятое место. Оба запроса соединены union all
query_5_1 = f'''
    (select
        e.name employer,
        count(*) vac_cnt
    from
        vacancies v
    join employers e on v.employer_id = e.id
    group by e.name
    order by vac_cnt desc
    limit 1)
    
    union all
    
    (select
        e.name employer,
        count(*) vac_cnt
    from
        vacancies v
    join employers e on v.employer_id = e.id
    group by e.name
    order by vac_cnt desc
    limit 1 offset 4)
'''

In [356]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
display(df)

,employer,vac_cnt
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.

In [357]:
# Запрос выводит количество работодателей и вакансий в каждом регионе.
query_5_2 = f'''
    select
        a.name region,
        count(e.name) employers_count,
        count(v.name) vacancies_count
    from
        areas a
    left join employers e on a.id = e.area
    left join vacancies v on v.area_id = a.id
    where v.id is null
    group by a.name, v.name
    order by count(e.name) desc
    limit 1
'''

In [358]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
display(df)

,region,employers_count,vacancies_count
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [359]:
# Запрос выводит количество регионов и вакансии в нем для представленной компании с помощью count(), далее группировка group by() и сортировка order by() 
query_5_3 = f'''
    select
        e.name employers,
        count(distinct v.area_id) areas_count
    from
        employers e
    left join vacancies v on e.id = v.employer_id
    group by e.name
    order by 2 desc
'''

In [360]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
display(df)

,employers,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [361]:
# Запрос выводит количество работодателей не указавших сферу деятельности, с помощью count()
query_5_4 = f'''
    select
        count(e.id) employers_without_industry_count
    from
        employers e
    left join employers_industries ei on e.id = ei.employer_id
    where ei.employer_id is null
'''

In [362]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
display(df)

,employers_without_industry_count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [363]:
# Запрос выводящий название компании, на третьем месте у которой указано четыре сферы деятельности
query_5_5 = f'''
    select
        e.name company_name
    from
        employers e
    left join employers_industries ei on e.id = ei.employer_id
    left join industries i on ei.industry_id = i.id
    group by e.name
    having count(distinct i.name) = 4
    order by e.name
    limit 1 offset 2
'''

In [364]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
display(df)

,company_name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [365]:
# Запрос выводит количество компаний со сферой деятельности - разработка ПО
query_5_6 = f'''
    select
        count(*) developers_count
    from
        employers e
    left join employers_industries ei on e.id = ei.employer_id
    left join industries i on ei.industry_id = i.id
    where i.name = 'Разработка программного обеспечения'
'''

In [366]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
display(df)

,developers_count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [367]:
import requests
from bs4 import BeautifulSoup

# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'

cities = [] # Cписок городов-миллионников

response = requests.get(url) # Загрузка HTML страницы по ссылке url
page = BeautifulSoup(response.text, 'html.parser') # Библиотека используется для парсинга HTML страницы
try:
    for row in page.find_all('tr')[1:]: # Проходим по строкам таблицы, пропуская первую строку заголовка
        columns = row.find_all('td')
        city_name = columns[1].get_text(strip=True) # Получаем название города из второго столбца, убираем пробелы
        cities.append(city_name)
except(IndexError): # Ловим ошибку которая тут возникает в конце
    pass
finally:
    cities_result = tuple(cities) # Конвертируем в кортеж

In [368]:
# Запрос выводит список городов-миллионников с количеством вакансий в них компании Яндекс.
# Первая часть запроса выводит список городов и количество вакансий, города идут по ссылке из кортежа (tuple) из ячейки выше
# Вторая часть (после union all) выводит строку TOTAL с суммой всех вакансий. C помощью union all все запросы склеиваются.
query_5_7 = f'''
    with yandex_jobs as (
        select
            a.name as million_plus_cities,
            count(v.id) as vacancies_count
        from
            vacancies v
        left join employers e on v.employer_id = e.id
        left join areas a on a.id = v.area_id
        where e.name = 'Яндекс'
        and a.name in {cities_result}
        group by a.name
        order by vacancies_count
        )
    select * 
    from 
    yandex_jobs
    
    union all 
    
    select
        'TOTAL',
        sum(vacancies_count)
    from
    yandex_jobs
'''

In [369]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
display(df)

,million_plus_cities,vacancies_count
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

# Выводы:

&ensp; Согласно данным, на первом месте, по количеству предлагаемых вакансий на рынке труда занимает компания "Яндекс", что указывает на значительную активность компании в поиске новых сотрудников. Пятерку основных работодателей закрывает компания "Газпром нефть", ее активность значительно ниже по сравнению с Яндексом. Эти данные могут свидетельствовать об основных направлениях данных компаний. Яндекс специализируется в сфере разработки ПО, беспилотных такси, онлайн сервисов. Газпром нефть больше специализируется на нефтяной отрасли и на технологиях по ее добычи. Видимо онлайн и IT сферы на данный момент более активно развиваются.

&ensp; Яндекс и Ростелеком имеют широкий географический охват, предлагая вакансии в большом количестве регионов. Спецремонт также имеет значительное географическое покрытие, но немного уступает лидерам.

&ensp; Существует значительное количество работодателей в регионах России, где нет активных вакансий, что может указывать либо на временное отсутствие открытых вакансий, либо на специфические потребности в трудовой силе, не удовлетворяемые текущими кандидатами.

&ensp; Сфера разработки ПО является одной из значительных отраслей на рынке труда, представленная большим количеством компаний. Это указывает на высокий спрос на IT-специалистов и активное развитие технологий.

&ensp; Корпорация "Яндекс", как один из лидеров рынка России и СНГ предоставляет свои вакансии в крупнейших городах России с численностью более миллиона жителей. Ожидаемо, что Москва, Санкт-Петербург, а также Екатеринбург, Нижний Новгород и Новосибирск являются для "Яндекс" наиболее привлекательными с точки зрения набора персонала для своих проектов.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [370]:
# Запрос выводит количество вакансий относящихся к данным
query_6_1 = f'''
    select
        count(*) data_vacancies_count
    from
        vacancies v
    where lower(v.name) like '%data%' 
    or lower(v.name) like '%данн%'
'''

In [371]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
display(df)

,data_vacancies_count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [372]:
# Запрос выводит количество вакансий по направлению Data Science для Junior, без опыта и со стажировкой
query_6_2 = f'''
    select
        count(*) junior_ds_vacancies_count
    from
        vacancies v
    where
    (
        lower(v.name) like '%data scientist%'
        or lower(v.name) like '%data science%'
        or lower(v.name) like '%machine learning%'
        or lower(v.name) like '%ml%' and lower(v.name) not like '%html%'
        or lower(v.name) like '%исследователь данных%'
        or lower(v.name) like '%машинн%обучен%'
    ) 
    and 
    (
        lower(v.name) like '%junior%'
        or lower(v.experience) like '%нет опыта%'
        or lower(v.employment) like '%стажировка%'
    )
'''

In [373]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
display(df)

,junior_ds_vacancies_count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [374]:
# Запрос выводит количество вакансий по направлению Data Science со знанием SQL, Postgres
query_6_3 = f'''
    select
        count(*) vacancies_with_sql_postgres_count
    from
        vacancies v
    where
    (
        lower(v.name) like '%data scientist%'
        or lower(v.name) like '%data science%'
        or lower(v.name) like '%machine learning%'
        or lower(v.name) like '%ml%' and lower(v.name) not like '%html%'
        or lower(v.name) like '%исследователь данных%'
        or lower(v.name) like '%машинн%обучен%'
    ) 
    and
    (
        lower(v.key_skills) like '%sql%'
        or lower(v.key_skills) like '%postgres%'
    )
'''

In [375]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
display(df)

,vacancies_with_sql_postgres_count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [376]:
# Запрос выводит количество вакансий по направлению Data Science со знанием Python
query_6_4 = f'''
    select
        count(*) vacancies_with_python_count
    from
        vacancies v
    where
    (
        lower(v.name) like '%data scientist%'
        or lower(v.name) like '%data science%'
        or lower(v.name) like '%machine learning%'
        or lower(v.name) like '%ml%' and lower(v.name) not like '%html%'
        or lower(v.name) like '%исследователь данных%'
        or lower(v.name) like '%машинн%обучен%'
    ) 
    and
    (
        lower(v.key_skills) like '%python%'
    )
'''

In [377]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
display(df)

,vacancies_with_python_count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [378]:
# Запрос выводит среднее количество ключевых навыков указанных в вакансиях для Data Science
# - round(..., 2) округляет число до 2 знаков после запятой;
# - avg(... + 1) считает среднее число;
# - length(v.key_skills) - length(replace(v.key_skills, CHR(9), '') - 
# это выражение подсчитывает количество табуляций CHR(9) или \t в строке key_skills и добавляет 1, 
# чтобы получить количество навыков.
query_6_5 = f'''
    select
        round(avg(length(v.key_skills) - length(replace(v.key_skills, CHR(9), '')) + 1), 2) average_key_skills_count
    from
        vacancies v
    where
    (
        lower(v.name) like '%data scientist%'
        or lower(v.name) like '%data science%'
        or lower(v.name) like '%machine learning%'
        or (v.name like '%ML%' and lower(v.name) not like '%html%')
        or lower(v.name) like '%исследователь данных%'
        or lower(v.name) like '%машинн%обучен%'
    )
    and
    (
        v.key_skills is not null
    )
'''

In [379]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
display(df)

,average_key_skills_count
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [380]:
# Запрос выводит среднее количество ключевых навыков указанных в вакансиях для Data Science
# - round() округляет число;
# - avg() считает среднее число;
# - coalesce((v.salary_from + v.salary_to) / 2.0, v.salary_from, v.salary_to, 0)) - 
# это выражение подсчитывает cумму верхней и нижней зарплатной вилки деля на 2, 
# если одно поле имеет null то заменяем его на 0
query_6_6 = f'''
    select
        v.experience,
        round(avg(coalesce((v.salary_from + v.salary_to) / 2.0, v.salary_from, v.salary_to, 0)), 0) average_salary
    from
        vacancies v
    where
    (
        lower(v.name) like '%data scientist%'
        or lower(v.name) like '%data science%'
        or lower(v.name) like '%machine learning%'
        or (v.name like '%ML%' and lower(v.name) not like '%html%')
        or lower(v.name) like '%исследователь данных%'
        or lower(v.name) like '%машинн%обучен%'
    )
    and 
    (
        v.salary_from is not null 
        or v.salary_to is not null
    )
    group by v.experience
    order by 1 desc
'''

In [381]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
display(df)

,experience,average_salary
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

# Выводы:

&ensp; В связи с полученными данными, можно сделать некоторые промежуточные выводы. Сфера Data Science активно развивается, что подтверждается значительным количеством вакансий на рынке труда. Это указывает на высокий спрос на специалистов в этой области.
Однако, несмотря на общий высокий спрос на специалистов в Data Science, количество позиций для начинающих специалистов, стажеров или кандидатов без опыта работы значительно меньше. Это может указывать на либо высокие требования к кандидатам, либо предпочтение опытных специалистов.  

&ensp; SQL и PostgreSQL являются востребованными навыками в данном направлении и требуются в значительном количестве вакансий. Python еще более востребован, что указывает на его ключевую роль в сфере Data Science.

&ensp; Среднее количество ключевых навыков, указываемых в вакансиях (6.41), демонстрирует, что работодатели ожидают от кандидатов владения широким спектром навыков.

&ensp; Средняя зарплата существенно зависит от опыта работы, что подтверждается полученными данными. Специалисты с опытом от 3 до 6 лет получают значительно больше, чем специалисты с опытом от 1 года до 3 лет и те, кто не имеет опыта. Эта разница в зарплате указывает на высокую ценность опыта в сфере Data Science и значительный рост заработной платы по мере накопления опыта.

# Общий вывод по проекту

На данный момент, обладая текущими данными можно сделать следующие утверждения:
- Рынок труда активен с большим количеством вакансий и работодателей. Значительное географическое покрытие и разнообразие сфер деятельности свидетельствуют о широких возможностях для соискателей;

- Основные экономические центры, такие как Москва и Санкт-Петербург, лидируют по количеству вакансий, что ожидаемо и подтверждают свой статус данными. Другие крупные города также демонстрируют значительный спрос на рабочую силу;

- Разнообразие предложений на рынке труда отражается в значительном диапазоне зарплат, что может зависеть от уровня квалификации и специфики вакансий;

- Традиционный рабочий день все еще преобладает, но удаленная работа и гибкий график также набирают популярность, предоставляя больше возможностей для соискателей и меньшие финансовые затраты для работодателя;

- Спрос на специалистов с опытом рыботы от 1 года до 6 лет наиболее высок. Рынок также предоставляет значительные возможности для начинающих специалистов;

- Data Science активно развивается, однако требует от кандидатов значительного набора опыта и знаний, что отражается в высокой зарплате для специалистов с опытом;

- Крупные компании активно участвуют на рынке труда и имеют широкий географический охват, предлагая множетво вакансий в разных регионах;

- Сфера разработки ПО активно развивается, подтверждая высокую потребность в IT-специалистах.

Соискателям следует развивать необходимые навыки требующиеся на рынке труда (особенно в IT секторе), набираться опыта для того, чтобы претендовать на более высокую заработную плату.

Работодателям стоит сделать описание вакансий и сфер деятельности более прозрачными, чтобы облегчить поиск для соискателей. Рассмотреть возможность создания программ для начинающих специалистов и стажировок в IT секторе.

# Дополнительно можно провести:
- Частотный анализ ключевых навыков: Исследовать наиболее часто упоминаемые навыки в вакансиях, чтобы определить, какие навыки наиболее востребованы на рынке труда.

- Распределение вакансий по регионам: Анализировать плотность вакансий в различных регионах, чтобы понять, где находится наибольший спрос на рабочую силу.

- Зарплаты по отраслям: Сравнить средние зарплаты в различных отраслях, чтобы выявить наиболее и наименее оплачиваемые сферы.

## <center> Частотный анализ ключевых навыков </center>

In [382]:
# Создаем запрос для получения ключевых навыков
query_vacancies = f'''
    select
        v.key_skills key_skills
    from
        vacancies v
'''

# 1. Загрузка данных, из созданного запроса
df = pd.read_sql_query(query_vacancies, connection)

# 2. Подготовка данных
# Разделяем ключевые навыки и проводим частотный анализ, c помощью:
# - dropna(): удаляем строки с пропусками в колонке key_skills;
# - str.split('\t'): разделяем ключевые навыки, перечисленные через cимвол табуляции;
# - explode(): разворачиваем списки навыков в отдельные строки;
# - str.strip(): удаляем лишние пробелы вокруг названий навыков.
skills_series = df['key_skills'].dropna().str.split('\t').explode().str.strip()
# Создаем DataFrame для частотного анализа
skills_frequency = skills_series.value_counts().reset_index()
skills_frequency.columns = ['skill', 'count']

# 3. Визуализация c помощью библиотеки plotly.express
fig_bar = px.bar(
    skills_frequency.head(20),
    x='skill', 
    y='count',
    title='Топ 20 ключевых навыков',
    width=1000, 
    height=400,
    labels={'skill': 'Навык', 'count': 'Частота'})

fig_bar.show()

# fig_hist.write_image("images/graphic_1.png", engine="kaleido");

<center> <img src = images/graphic_1.png alt="drawing" style="width:1000px; heigth:400px;">

# Вывод:
На графике выше видно, что среди 20 самых востребованных навыков среди всех вакансий является знание SQL, Git и английский язык.

## <center> Распределение вакансий по регионам </center>

In [383]:
# Создаем запрос для получения вакансий
query_vacancies_by_areas = f'''
    select
        a.name region,
        v.name vacancies
    from
        vacancies v
    left join areas a on v.area_id = a.id
'''

# 1. Загрузка данных, из созданного запроса
df = pd.read_sql_query(query_vacancies_by_areas, connection)

# 2. Подготовка данных, подсчитываем количество вакансий по регионам
vacancies_series = df['region'].value_counts().reset_index()

# 3. Визуализация c помощью библиотеки plotly.express
fig_bar = px.bar(
    vacancies_series.head(30),
    x='region',
    y='count',
    color='region',
    title='Распределение вакансий по регионам (топ 30 регионов)',
    width=1000, 
    height=400,
    labels={'region': 'Регион', 'count': 'Количество вакансий'})

fig_bar.show()

# fig_hist.write_image("images/graphic_2.png", engine="kaleido");

<center> <img src = images/graphic_2.png alt="drawing" style="width:1000px; heigth:400px;">

## Вывод:
Ранее сделанные выводы относительно наиболее активных рынков труда является: Москва, Санкт-Петербург и Минск.a

## <center> Зарплаты по отраслям </center>

In [384]:
# Создаем запрос для получения средней заработной платы по отраслям
query_salary_by_industries = f'''
    select
        round(avg(coalesce((v.salary_from + v.salary_to) / 2.0, v.salary_from, v.salary_to, 0)), 0) average_salary,
        i.name industries
    from
        vacancies v
    join employers e on v.employer_id = e.id
    join employers_industries ei on ei.employer_id = e.id
    join industries i on ei.industry_id = i.id
    where v.salary_from is not null or v.salary_to is not null
    group by i.name
    order by average_salary desc
'''

# 1. Загрузка данных, из созданного запроса
df = pd.read_sql_query(query_salary_by_industries, connection)

# 2. Подготовка данных, cокращение длинных строк
df['short_industries'] = df['industries'].str.slice(0, 15) # Обрезка строк до 15 символов
salary_data = df

# 3. Визуализация c помощью библиотеки plotly.express
fig_hist = px.histogram(
    salary_data.head(30),
    y='average_salary',
    x='short_industries',
    title='Зарплаты по отраслям',
    nbins=300,
    width=1000,
    height=400,
    labels={'short_industries': 'Отрасль (сокращенно)', 'average_salary': 'Средняя зарплата (руб)'})

fig_hist.update_layout(xaxis_tickangle=-45)

fig_hist.show()

# fig_hist.write_image("images/graphic_3.png", engine="kaleido");

<center> <img src = images/graphic_3.png alt="drawing" style="width:1000px; heigth:400px;">

# Вывод:
Интересные результаты получились по средним зарплатам и отраслям. Самой высокооплачиваемой отраслью судя по графику является Агрохимия, где средняя заработная плата составляет 381 тыс.руб.

In [385]:
# Закрываем соединение с БД
connection.close()